In [ ]:
import pandas as pd
from numpy import *
import numpy as np
from KGD_algorithm_by_Hold_out_L2_infinity_norm import generate_data, Test_error_KGD_HO
import math
import h5py
from sklearn.model_selection import KFold
import os, time
from datetime import datetime
time_start = time.time()
import random
import matplotlib.pyplot as plt
start_time = datetime.now()
start_time_str = start_time.strftime('%Y.%m.%d')

noise_var = 0.6
split_L_per = 1.0 # 💎1， 0.8，0.6， 0.4

dim = 1
step_size = 1

log_filename = f'./HO_result_10trials/HOtest_1000_dim{dim}_noisevar{noise_var}_stepsize{step_size}_start{start_time_str}_log.txt'
# 初始化日志文件
with open(log_filename, 'w') as log_file:
    log_file.write("Start Logging\n")

L2_t_list_20trial, L2_norm_list_20trial = np.empty(shape=(10, 1)), np.empty(shape=(10, 1))
Linfinity_t_list_20trial, Linfinity_norm_list_20trial = np.empty(shape=(10, 1)), np.empty(shape=(10, 1))
L2_norm_list_20trial_mean, Linfinity_norm_list_20trial_mean = [], []

for size_i in range(1,2):
    L2_t_star_list, L2_test_error_list, Linfinity_t_star_list, Linfinity_test_error_list = [], [], [], []
    train_size = 1000 * size_i
    split_L = int(train_size * split_L_per)
    train, test = (train_size, dim), (200, dim)
    print(' ------------------  💎 dim: %s; step_size: %s; train_size: %s; split_L_per: %s; noise_var: %s  ------------------' % (dim, step_size, train_size, split_L_per, noise_var))

    for trial_i in range(15, 25):
        print('------------------   trial_i: %s  ------------------' % (trial_i-9))
        np.random.seed(trial_i)
        _, _, X_test, y_test = generate_data((300, dim), (500, dim), dim, noise_var)  #保证 test只在不同的trial时有所不同
        X_train, y_train, _, _ = generate_data(train, test, dim, noise_var)
        L2_t_star, L2_test_error, Linfinity_t_star, Linfinity_test_error = Test_error_KGD_HO(X_train, y_train, X_test, y_test, dim, split_L, step_size)
        L2_t_star_list.append(L2_t_star)
        L2_test_error_list.append(L2_test_error)
        Linfinity_t_star_list.append(Linfinity_t_star)
        Linfinity_test_error_list.append(Linfinity_test_error)
        print('L2_t_star_list:', L2_t_star_list)
        print('L2_test_error_list:', L2_test_error_list)
        print('Linfinity_t_star_list:', Linfinity_t_star_list)
        print('Linfinity_test_error_list:', Linfinity_test_error_list)

        # 💎logging in process "for trial_i in range(10, 20)"
        with open(log_filename, 'a') as log_file:
            log_file.write(
                f'--------------------------- train_size: {train_size}; trial_i: {trial_i - 10}--------------------------- \n'
                f'history:\nL2_t_star_list: {L2_t_star_list}\n'
                f'L2_test_error_list: {L2_test_error_list}\n'
                f'Linfinity_t_star_list: {Linfinity_t_star_list}\n'
                f'Linfinity_test_error_list: {Linfinity_test_error_list}\n')

    L2_t_list_20trial[:, size_i - 1] = np.squeeze(np.array(L2_t_star_list))
    L2_norm_list_20trial[:, size_i - 1] = np.squeeze(np.array(L2_test_error_list))
    L2_test_error_list_mean = np.mean(L2_test_error_list)
    L2_norm_list_20trial_mean.append(L2_test_error_list_mean)

    Linfinity_t_list_20trial[:, size_i - 1] = np.squeeze(np.array(Linfinity_t_star_list))
    Linfinity_norm_list_20trial[:, size_i - 1] = np.squeeze(np.array(Linfinity_test_error_list))
    Linfinity_test_error_list_mean = np.mean(Linfinity_test_error_list)
    Linfinity_norm_list_20trial_mean.append(Linfinity_test_error_list_mean)


    # 💎logging in process "for trial_i in range(10, 20)"
    with open(log_filename, 'a') as log_file:
        log_file.write(
            f'------------------------------------------------- train_size: {train_size} ------------------------------------------------- \n'
            f'history:\nL2_t_list_20trial: {L2_t_list_20trial}\n'
            f'L2_norm_list_20trial: {L2_norm_list_20trial}\n'
            f'L2_norm_list_20trial_mean: {L2_norm_list_20trial_mean}\n'
            f'Linfinity_t_list_20trial: {Linfinity_t_list_20trial}\n'
            f'Linfinity_norm_list_20trial: {Linfinity_norm_list_20trial}\n'
            f'Linfinity_norm_list_20trial_mean: {Linfinity_norm_list_20trial_mean}\n'
        )

    # ☀️save in process
    KGD_result = {
        'L2_t_list_20trial': L2_t_list_20trial,
        'L2_norm_list_20trial': L2_norm_list_20trial,
        'L2_norm_list_20trial_mean': L2_norm_list_20trial_mean,
        'Linfinity_t_list_20trial': Linfinity_t_list_20trial,
        'Linfinity_norm_list_20trial': Linfinity_norm_list_20trial,
        'Linfinity_norm_list_20trial_mean': Linfinity_norm_list_20trial_mean}
    filename = f'./HO_result_10trials/HOtest_1000_dim{dim}_noisevar{noise_var}_stepsize{step_size}_new.npy'
    np.save(filename, KGD_result)


time_total = time.time() - time_start

# ☀️save final
KGD_result = {
    'L2_t_list_20trial': L2_t_list_20trial,
    'L2_norm_list_20trial': L2_norm_list_20trial,
    'L2_norm_list_20trial_mean': L2_norm_list_20trial_mean,
    'Linfinity_t_list_20trial': Linfinity_t_list_20trial,
    'Linfinity_norm_list_20trial': Linfinity_norm_list_20trial,
    'Linfinity_norm_list_20trial_mean': Linfinity_norm_list_20trial_mean}
filename = f'./HO_result_10trials/HOtest_1000_dim{dim}_noisevar{noise_var}_stepsize{step_size}_new.npy'
np.save(filename, KGD_result)
with open(log_filename, 'a') as log_file:
    log_file.write(f'Total running time: {time_total}\n')


print('------------------------------ runing information ------------------------------------------')
print('running time:', time_total)
print(filename)
print(KGD_result.keys())
print(log_filename)
print("Logging complete.")




In [ ]:
filename = f'./HO_result_10trials/HOtest_1000_dim{dim}_noisevar{noise_var}_stepsize{step_size}_new.npy'
LP_result = np.load(filename, allow_pickle=True).item()
LP_L2_norm_list_20trial_mean = LP_result['L2_norm_list_20trial_mean']
LP_Linfinity_norm_list_20trial_mean = LP_result['Linfinity_norm_list_20trial_mean']
np.set_printoptions(suppress=True, precision=6)
print(LP_L2_norm_list_20trial_mean)
print(LP_Linfinity_norm_list_20trial_mean)

In [ ]:
import pandas as pd
from numpy import *
import numpy as np
from KGD_algorithm_by_Hold_out_L2_infinity_norm import generate_data, Test_error_KGD_HO
import math
import h5py
from sklearn.model_selection import KFold
import os, time
from datetime import datetime
time_start = time.time()
import random
import matplotlib.pyplot as plt
start_time = datetime.now()
start_time_str = start_time.strftime('%Y.%m.%d')

noise_var = 0.6
split_L_per = 1.0 # 💎1， 0.8，0.6， 0.4

dim = 1
step_size = 1

log_filename = f'./HO_result_10trials/HOtest_1200_dim{dim}_noisevar{noise_var}_stepsize{step_size}_start{start_time_str}_log.txt'
# 初始化日志文件
with open(log_filename, 'w') as log_file:
    log_file.write("Start Logging\n")

L2_t_list_20trial, L2_norm_list_20trial = np.empty(shape=(10, 1)), np.empty(shape=(10, 1))
Linfinity_t_list_20trial, Linfinity_norm_list_20trial = np.empty(shape=(10, 1)), np.empty(shape=(10, 1))
L2_norm_list_20trial_mean, Linfinity_norm_list_20trial_mean = [], []

for size_i in range(1,2):
    L2_t_star_list, L2_test_error_list, Linfinity_t_star_list, Linfinity_test_error_list = [], [], [], []
    train_size = 1200 * size_i
    split_L = int(train_size * split_L_per)
    train, test = (train_size, dim), (200, dim)
    print(' ------------------  💎 dim: %s; step_size: %s; train_size: %s; split_L_per: %s; noise_var: %s  ------------------' % (dim, step_size, train_size, split_L_per, noise_var))

    for trial_i in range(15, 25):
        print('------------------   trial_i: %s  ------------------' % (trial_i-9))
        np.random.seed(trial_i)
        _, _, X_test, y_test = generate_data((300, dim), (500, dim), dim, noise_var)  #保证 test只在不同的trial时有所不同
        X_train, y_train, _, _ = generate_data(train, test, dim, noise_var)
        L2_t_star, L2_test_error, Linfinity_t_star, Linfinity_test_error = Test_error_KGD_HO(X_train, y_train, X_test, y_test, dim, split_L, step_size)
        L2_t_star_list.append(L2_t_star)
        L2_test_error_list.append(L2_test_error)
        Linfinity_t_star_list.append(Linfinity_t_star)
        Linfinity_test_error_list.append(Linfinity_test_error)
        print('L2_t_star_list:', L2_t_star_list)
        print('L2_test_error_list:', L2_test_error_list)
        print('Linfinity_t_star_list:', Linfinity_t_star_list)
        print('Linfinity_test_error_list:', Linfinity_test_error_list)

        # 💎logging in process "for trial_i in range(10, 20)"
        with open(log_filename, 'a') as log_file:
            log_file.write(
                f'--------------------------- train_size: {train_size}; trial_i: {trial_i - 10}--------------------------- \n'
                f'history:\nL2_t_star_list: {L2_t_star_list}\n'
                f'L2_test_error_list: {L2_test_error_list}\n'
                f'Linfinity_t_star_list: {Linfinity_t_star_list}\n'
                f'Linfinity_test_error_list: {Linfinity_test_error_list}\n')

    L2_t_list_20trial[:, size_i - 1] = np.squeeze(np.array(L2_t_star_list))
    L2_norm_list_20trial[:, size_i - 1] = np.squeeze(np.array(L2_test_error_list))
    L2_test_error_list_mean = np.mean(L2_test_error_list)
    L2_norm_list_20trial_mean.append(L2_test_error_list_mean)

    Linfinity_t_list_20trial[:, size_i - 1] = np.squeeze(np.array(Linfinity_t_star_list))
    Linfinity_norm_list_20trial[:, size_i - 1] = np.squeeze(np.array(Linfinity_test_error_list))
    Linfinity_test_error_list_mean = np.mean(Linfinity_test_error_list)
    Linfinity_norm_list_20trial_mean.append(Linfinity_test_error_list_mean)


    # 💎logging in process "for trial_i in range(10, 20)"
    with open(log_filename, 'a') as log_file:
        log_file.write(
            f'------------------------------------------------- train_size: {train_size} ------------------------------------------------- \n'
            f'history:\nL2_t_list_20trial: {L2_t_list_20trial}\n'
            f'L2_norm_list_20trial: {L2_norm_list_20trial}\n'
            f'L2_norm_list_20trial_mean: {L2_norm_list_20trial_mean}\n'
            f'Linfinity_t_list_20trial: {Linfinity_t_list_20trial}\n'
            f'Linfinity_norm_list_20trial: {Linfinity_norm_list_20trial}\n'
            f'Linfinity_norm_list_20trial_mean: {Linfinity_norm_list_20trial_mean}\n'
        )

    # ☀️save in process
    KGD_result = {
        'L2_t_list_20trial': L2_t_list_20trial,
        'L2_norm_list_20trial': L2_norm_list_20trial,
        'L2_norm_list_20trial_mean': L2_norm_list_20trial_mean,
        'Linfinity_t_list_20trial': Linfinity_t_list_20trial,
        'Linfinity_norm_list_20trial': Linfinity_norm_list_20trial,
        'Linfinity_norm_list_20trial_mean': Linfinity_norm_list_20trial_mean}
    filename = f'./HO_result_10trials/HOtest_1200_dim{dim}_noisevar{noise_var}_stepsize{step_size}_new.npy'
    np.save(filename, KGD_result)


time_total = time.time() - time_start

# ☀️save final
KGD_result = {
    'L2_t_list_20trial': L2_t_list_20trial,
    'L2_norm_list_20trial': L2_norm_list_20trial,
    'L2_norm_list_20trial_mean': L2_norm_list_20trial_mean,
    'Linfinity_t_list_20trial': Linfinity_t_list_20trial,
    'Linfinity_norm_list_20trial': Linfinity_norm_list_20trial,
    'Linfinity_norm_list_20trial_mean': Linfinity_norm_list_20trial_mean}
filename = f'./HO_result_10trials/HOtest_1200_dim{dim}_noisevar{noise_var}_stepsize{step_size}_new.npy'
np.save(filename, KGD_result)
with open(log_filename, 'a') as log_file:
    log_file.write(f'Total running time: {time_total}\n')


print('------------------------------ runing information ------------------------------------------')
print('running time:', time_total)
print(filename)
print(KGD_result.keys())
print(log_filename)
print("Logging complete.")




In [ ]:
filename = f'./HO_result_10trials/HOtest_1200_dim{dim}_noisevar{noise_var}_stepsize{step_size}_new.npy'
LP_result = np.load(filename, allow_pickle=True).item()
LP_L2_norm_list_20trial_mean = LP_result['L2_norm_list_20trial_mean']
LP_Linfinity_norm_list_20trial_mean = LP_result['Linfinity_norm_list_20trial_mean']
np.set_printoptions(suppress=True, precision=6)
print(LP_L2_norm_list_20trial_mean)
print(LP_Linfinity_norm_list_20trial_mean)

In [ ]:
import pandas as pd
from numpy import *
import numpy as np
from KGD_algorithm_by_Hold_out_L2_infinity_norm import generate_data, Test_error_KGD_HO
import math
import h5py
from sklearn.model_selection import KFold
import os, time
from datetime import datetime
time_start = time.time()
import random
import matplotlib.pyplot as plt
start_time = datetime.now()
start_time_str = start_time.strftime('%Y.%m.%d')

noise_var = 0.6
split_L_per = 1.0 # 💎1， 0.8，0.6， 0.4

dim = 3
step_size = 3

log_filename = f'./HO_result_10trials/HOtest_1000_dim{dim}_noisevar{noise_var}_stepsize{step_size}_start{start_time_str}_log.txt'
# 初始化日志文件
with open(log_filename, 'w') as log_file:
    log_file.write("Start Logging\n")

L2_t_list_20trial, L2_norm_list_20trial = np.empty(shape=(10, 1)), np.empty(shape=(10, 1))
Linfinity_t_list_20trial, Linfinity_norm_list_20trial = np.empty(shape=(10, 1)), np.empty(shape=(10, 1))
L2_norm_list_20trial_mean, Linfinity_norm_list_20trial_mean = [], []

for size_i in range(1,2):
    L2_t_star_list, L2_test_error_list, Linfinity_t_star_list, Linfinity_test_error_list = [], [], [], []
    train_size = 1000 * size_i
    split_L = int(train_size * split_L_per)
    train, test = (train_size, dim), (200, dim)
    print(' ------------------  💎 dim: %s; step_size: %s; train_size: %s; split_L_per: %s; noise_var: %s  ------------------' % (dim, step_size, train_size, split_L_per, noise_var))

    for trial_i in range(15, 25):
        print('------------------   trial_i: %s  ------------------' % (trial_i-9))
        np.random.seed(trial_i)
        _, _, X_test, y_test = generate_data((300, dim), (500, dim), dim, noise_var)  #保证 test只在不同的trial时有所不同
        X_train, y_train, _, _ = generate_data(train, test, dim, noise_var)
        L2_t_star, L2_test_error, Linfinity_t_star, Linfinity_test_error = Test_error_KGD_HO(X_train, y_train, X_test, y_test, dim, split_L, step_size)
        L2_t_star_list.append(L2_t_star)
        L2_test_error_list.append(L2_test_error)
        Linfinity_t_star_list.append(Linfinity_t_star)
        Linfinity_test_error_list.append(Linfinity_test_error)
        print('L2_t_star_list:', L2_t_star_list)
        print('L2_test_error_list:', L2_test_error_list)
        print('Linfinity_t_star_list:', Linfinity_t_star_list)
        print('Linfinity_test_error_list:', Linfinity_test_error_list)

        # 💎logging in process "for trial_i in range(10, 20)"
        with open(log_filename, 'a') as log_file:
            log_file.write(
                f'--------------------------- train_size: {train_size}; trial_i: {trial_i - 10}--------------------------- \n'
                f'history:\nL2_t_star_list: {L2_t_star_list}\n'
                f'L2_test_error_list: {L2_test_error_list}\n'
                f'Linfinity_t_star_list: {Linfinity_t_star_list}\n'
                f'Linfinity_test_error_list: {Linfinity_test_error_list}\n')

    L2_t_list_20trial[:, size_i - 1] = np.squeeze(np.array(L2_t_star_list))
    L2_norm_list_20trial[:, size_i - 1] = np.squeeze(np.array(L2_test_error_list))
    L2_test_error_list_mean = np.mean(L2_test_error_list)
    L2_norm_list_20trial_mean.append(L2_test_error_list_mean)

    Linfinity_t_list_20trial[:, size_i - 1] = np.squeeze(np.array(Linfinity_t_star_list))
    Linfinity_norm_list_20trial[:, size_i - 1] = np.squeeze(np.array(Linfinity_test_error_list))
    Linfinity_test_error_list_mean = np.mean(Linfinity_test_error_list)
    Linfinity_norm_list_20trial_mean.append(Linfinity_test_error_list_mean)


    # 💎logging in process "for trial_i in range(10, 20)"
    with open(log_filename, 'a') as log_file:
        log_file.write(
            f'------------------------------------------------- train_size: {train_size} ------------------------------------------------- \n'
            f'history:\nL2_t_list_20trial: {L2_t_list_20trial}\n'
            f'L2_norm_list_20trial: {L2_norm_list_20trial}\n'
            f'L2_norm_list_20trial_mean: {L2_norm_list_20trial_mean}\n'
            f'Linfinity_t_list_20trial: {Linfinity_t_list_20trial}\n'
            f'Linfinity_norm_list_20trial: {Linfinity_norm_list_20trial}\n'
            f'Linfinity_norm_list_20trial_mean: {Linfinity_norm_list_20trial_mean}\n'
        )

    # ☀️save in process
    KGD_result = {
        'L2_t_list_20trial': L2_t_list_20trial,
        'L2_norm_list_20trial': L2_norm_list_20trial,
        'L2_norm_list_20trial_mean': L2_norm_list_20trial_mean,
        'Linfinity_t_list_20trial': Linfinity_t_list_20trial,
        'Linfinity_norm_list_20trial': Linfinity_norm_list_20trial,
        'Linfinity_norm_list_20trial_mean': Linfinity_norm_list_20trial_mean}
    filename = f'./HO_result_10trials/HOtest_1000_dim{dim}_noisevar{noise_var}_stepsize{step_size}_new.npy'
    np.save(filename, KGD_result)


time_total = time.time() - time_start

# ☀️save final
KGD_result = {
    'L2_t_list_20trial': L2_t_list_20trial,
    'L2_norm_list_20trial': L2_norm_list_20trial,
    'L2_norm_list_20trial_mean': L2_norm_list_20trial_mean,
    'Linfinity_t_list_20trial': Linfinity_t_list_20trial,
    'Linfinity_norm_list_20trial': Linfinity_norm_list_20trial,
    'Linfinity_norm_list_20trial_mean': Linfinity_norm_list_20trial_mean}
filename = f'./HO_result_10trials/HOtest_1000_dim{dim}_noisevar{noise_var}_stepsize{step_size}_new.npy'
np.save(filename, KGD_result)
with open(log_filename, 'a') as log_file:
    log_file.write(f'Total running time: {time_total}\n')


print('------------------------------ runing information ------------------------------------------')
print('running time:', time_total)
print(filename)
print(KGD_result.keys())
print(log_filename)
print("Logging complete.")




In [ ]:
filename = f'./HO_result_10trials/HOtest_1000_dim{dim}_noisevar{noise_var}_stepsize{step_size}_new.npy'
LP_result = np.load(filename, allow_pickle=True).item()
LP_L2_norm_list_20trial_mean = LP_result['L2_norm_list_20trial_mean']
LP_Linfinity_norm_list_20trial_mean = LP_result['Linfinity_norm_list_20trial_mean']
np.set_printoptions(suppress=True, precision=6)
print(LP_L2_norm_list_20trial_mean)
print(LP_Linfinity_norm_list_20trial_mean)

In [ ]:
import pandas as pd
from numpy import *
import numpy as np
from KGD_algorithm_by_Hold_out_L2_infinity_norm import generate_data, Test_error_KGD_HO
import math
import h5py
from sklearn.model_selection import KFold
import os, time
from datetime import datetime
time_start = time.time()
import random
import matplotlib.pyplot as plt
start_time = datetime.now()
start_time_str = start_time.strftime('%Y.%m.%d')

noise_var = 0.6
split_L_per = 1.0 # 💎1， 0.8，0.6， 0.4

dim = 3
step_size = 3

log_filename = f'./HO_result_10trials/HOtest_1200_dim{dim}_noisevar{noise_var}_stepsize{step_size}_start{start_time_str}_log.txt'
# 初始化日志文件
with open(log_filename, 'w') as log_file:
    log_file.write("Start Logging\n")

L2_t_list_20trial, L2_norm_list_20trial = np.empty(shape=(10, 1)), np.empty(shape=(10, 1))
Linfinity_t_list_20trial, Linfinity_norm_list_20trial = np.empty(shape=(10, 1)), np.empty(shape=(10, 1))
L2_norm_list_20trial_mean, Linfinity_norm_list_20trial_mean = [], []

for size_i in range(1,2):
    L2_t_star_list, L2_test_error_list, Linfinity_t_star_list, Linfinity_test_error_list = [], [], [], []
    train_size = 1200 * size_i
    split_L = int(train_size * split_L_per)
    train, test = (train_size, dim), (200, dim)
    print(' ------------------  💎 dim: %s; step_size: %s; train_size: %s; split_L_per: %s; noise_var: %s  ------------------' % (dim, step_size, train_size, split_L_per, noise_var))

    for trial_i in range(15, 25):
        print('------------------   trial_i: %s  ------------------' % (trial_i-9))
        np.random.seed(trial_i)
        _, _, X_test, y_test = generate_data((300, dim), (500, dim), dim, noise_var)  #保证 test只在不同的trial时有所不同
        X_train, y_train, _, _ = generate_data(train, test, dim, noise_var)
        L2_t_star, L2_test_error, Linfinity_t_star, Linfinity_test_error = Test_error_KGD_HO(X_train, y_train, X_test, y_test, dim, split_L, step_size)
        L2_t_star_list.append(L2_t_star)
        L2_test_error_list.append(L2_test_error)
        Linfinity_t_star_list.append(Linfinity_t_star)
        Linfinity_test_error_list.append(Linfinity_test_error)
        print('L2_t_star_list:', L2_t_star_list)
        print('L2_test_error_list:', L2_test_error_list)
        print('Linfinity_t_star_list:', Linfinity_t_star_list)
        print('Linfinity_test_error_list:', Linfinity_test_error_list)

        # 💎logging in process "for trial_i in range(10, 20)"
        with open(log_filename, 'a') as log_file:
            log_file.write(
                f'--------------------------- train_size: {train_size}; trial_i: {trial_i - 10}--------------------------- \n'
                f'history:\nL2_t_star_list: {L2_t_star_list}\n'
                f'L2_test_error_list: {L2_test_error_list}\n'
                f'Linfinity_t_star_list: {Linfinity_t_star_list}\n'
                f'Linfinity_test_error_list: {Linfinity_test_error_list}\n')

    L2_t_list_20trial[:, size_i - 1] = np.squeeze(np.array(L2_t_star_list))
    L2_norm_list_20trial[:, size_i - 1] = np.squeeze(np.array(L2_test_error_list))
    L2_test_error_list_mean = np.mean(L2_test_error_list)
    L2_norm_list_20trial_mean.append(L2_test_error_list_mean)

    Linfinity_t_list_20trial[:, size_i - 1] = np.squeeze(np.array(Linfinity_t_star_list))
    Linfinity_norm_list_20trial[:, size_i - 1] = np.squeeze(np.array(Linfinity_test_error_list))
    Linfinity_test_error_list_mean = np.mean(Linfinity_test_error_list)
    Linfinity_norm_list_20trial_mean.append(Linfinity_test_error_list_mean)


    # 💎logging in process "for trial_i in range(10, 20)"
    with open(log_filename, 'a') as log_file:
        log_file.write(
            f'------------------------------------------------- train_size: {train_size} ------------------------------------------------- \n'
            f'history:\nL2_t_list_20trial: {L2_t_list_20trial}\n'
            f'L2_norm_list_20trial: {L2_norm_list_20trial}\n'
            f'L2_norm_list_20trial_mean: {L2_norm_list_20trial_mean}\n'
            f'Linfinity_t_list_20trial: {Linfinity_t_list_20trial}\n'
            f'Linfinity_norm_list_20trial: {Linfinity_norm_list_20trial}\n'
            f'Linfinity_norm_list_20trial_mean: {Linfinity_norm_list_20trial_mean}\n'
        )

    # ☀️save in process
    KGD_result = {
        'L2_t_list_20trial': L2_t_list_20trial,
        'L2_norm_list_20trial': L2_norm_list_20trial,
        'L2_norm_list_20trial_mean': L2_norm_list_20trial_mean,
        'Linfinity_t_list_20trial': Linfinity_t_list_20trial,
        'Linfinity_norm_list_20trial': Linfinity_norm_list_20trial,
        'Linfinity_norm_list_20trial_mean': Linfinity_norm_list_20trial_mean}
    filename = f'./HO_result_10trials/HOtest_1200_dim{dim}_noisevar{noise_var}_stepsize{step_size}_new.npy'
    np.save(filename, KGD_result)


time_total = time.time() - time_start

# ☀️save final
KGD_result = {
    'L2_t_list_20trial': L2_t_list_20trial,
    'L2_norm_list_20trial': L2_norm_list_20trial,
    'L2_norm_list_20trial_mean': L2_norm_list_20trial_mean,
    'Linfinity_t_list_20trial': Linfinity_t_list_20trial,
    'Linfinity_norm_list_20trial': Linfinity_norm_list_20trial,
    'Linfinity_norm_list_20trial_mean': Linfinity_norm_list_20trial_mean}
filename = f'./HO_result_10trials/HOtest_1200_dim{dim}_noisevar{noise_var}_stepsize{step_size}_new.npy'
np.save(filename, KGD_result)
with open(log_filename, 'a') as log_file:
    log_file.write(f'Total running time: {time_total}\n')


print('------------------------------ runing information ------------------------------------------')
print('running time:', time_total)
print(filename)
print(KGD_result.keys())
print(log_filename)
print("Logging complete.")




In [ ]:
filename = f'./HO_result_10trials/HOtest_1200_dim{dim}_noisevar{noise_var}_stepsize{step_size}_new.npy'
LP_result = np.load(filename, allow_pickle=True).item()
LP_L2_norm_list_20trial_mean = LP_result['L2_norm_list_20trial_mean']
LP_Linfinity_norm_list_20trial_mean = LP_result['Linfinity_norm_list_20trial_mean']
np.set_printoptions(suppress=True, precision=6)
print(LP_L2_norm_list_20trial_mean)
print(LP_Linfinity_norm_list_20trial_mean)

In [ ]:
import pandas as pd
from numpy import *
import numpy as np
from HSS_test_compare_BP import generate_data, Test_error_KGD_BSP
import math
import h5py
from sklearn.model_selection import KFold
import os, time
from datetime import datetime
time_start = time.time()
import random
import matplotlib.pyplot as plt
start_time = datetime.now()
start_time_str = start_time.strftime('%Y.%m.%d')

split_L_tr_per = 0.7 # 固定不变
noise_var = 0.6



dim = 1
step_size = 1
split_L_per = 0.6 # 💎1， 0.8，0.6， 0.4
C_bsp_list = [0.00001, 0.0001, 0.001, 0.01] + [i * 0.05 for i in range(1, 21, 1)]  # d=1

log_filename = f'./HSS_result_10trials/Small_Crange01new_HSStest_1000_dim{dim}_noisevar{noise_var}_split_L_per{split_L_per}_stepsize{step_size}_start{start_time_str}_log.txt'
# 初始化日志文件
with open(log_filename, 'w') as log_file:
    log_file.write("Start Logging\n")


# L2_C_20trial, L2_t_list_20trial, L2_norm_list_20trial = np.empty(shape=(10, 15)), np.empty(shape=(10, 15)), np.empty(shape=(10, 15))
# Linfinity_C_20trial, Linfinity_t_list_20trial, Linfinity_norm_list_20trial = np.empty(shape=(10, 15)), np.empty(shape=(10, 15)), np.empty(shape=(10, 15))
L2_C_20trial, L2_t_list_20trial, L2_norm_list_20trial = np.empty(shape=(10, 1)), np.empty(shape=(10, 1)), np.empty(shape=(10, 1))
Linfinity_C_20trial, Linfinity_t_list_20trial, Linfinity_norm_list_20trial = np.empty(shape=(10, 1)), np.empty(shape=(10, 1)), np.empty(shape=(10, 1))
L2_norm_list_20trial_mean, Linfinity_norm_list_20trial_mean = [], []


for size_i in range(1,2):
    L2_C_list, L2_t_star_list, L2_test_error_list, Linfinity_C_list, Linfinity_t_star_list, Linfinity_test_error_list = [], [], [], [], [], []
    train_size = 1000 * size_i
    split_L = int(train_size * split_L_per)
    split_L_tr = int(split_L * split_L_tr_per)
    train, test = (train_size, dim), (200, dim)
    print(' ------------------  💎 dim: %s; step_size: %s; train_size: %s; split_L_per: %s; split_L_tr_per: %s; noise_var: %s  ------------------' % (dim, step_size, train_size, split_L_per, split_L_tr_per, noise_var))

    for trial_i in range(15, 25):
        print('------------------   trial_i: %s  ------------------' % (trial_i-9))
        np.random.seed(trial_i)
        _, _, X_test, y_test = generate_data((300, dim), (500, dim), dim, noise_var)  #保证 test只在不同的trial时有所不同
        X_train, y_train, _, _ = generate_data(train, test, dim, noise_var)
        L2_C, L2_t_star, L2_test_error, Linfinity_C, Linfinity_t_star, Linfinity_test_error = Test_error_KGD_BSP(X_train, y_train, X_test, y_test, dim, step_size, split_L, split_L_tr, log_filename, C_bsp_list)
        L2_C_list.append(L2_C)
        L2_t_star_list.append(L2_t_star)
        L2_test_error_list.append(L2_test_error)
        Linfinity_C_list.append(Linfinity_C)
        Linfinity_t_star_list.append(Linfinity_t_star)
        Linfinity_test_error_list.append(Linfinity_test_error)
        print('L2_t_star_list:', L2_t_star_list)
        print('L2_test_error_list:', L2_test_error_list)
        print('Linfinity_t_star_list:', Linfinity_t_star_list)
        print('Linfinity_test_error_list:', Linfinity_test_error_list)


        # 💎logging in process "for trial_i in range(10, 20)"
        with open(log_filename, 'a') as log_file:
            log_file.write(
                f'--------------------------- train_size: {train_size}; trial_i: {trial_i-10}--------------------------- \n'
                f'history:\nL2_C_list: {L2_C_list}\n'
                f'L2_t_star_list: {L2_t_star_list}\n'
                f'L2_test_error_list: {L2_test_error_list}\n'
                f'Linfinity_C_list: {Linfinity_C_list}\n'
                f'Linfinity_t_star_list: {Linfinity_t_star_list}\n'
                f'Linfinity_test_error_list: {Linfinity_test_error_list}\n')


    L2_C_20trial[:, size_i-1] = np.squeeze(np.array(L2_C_list))
    L2_t_list_20trial[:, size_i - 1] = np.squeeze(np.array(L2_t_star_list))
    L2_norm_list_20trial[:, size_i - 1] = np.squeeze(np.array(L2_test_error_list))
    L2_test_error_list_mean = np.mean(L2_test_error_list)
    L2_norm_list_20trial_mean.append(L2_test_error_list_mean)

    Linfinity_C_20trial[:, size_i-1] = np.squeeze(np.array(Linfinity_C_list))
    Linfinity_t_list_20trial[:, size_i - 1] = np.squeeze(np.array(Linfinity_t_star_list))
    Linfinity_norm_list_20trial[:, size_i - 1] = np.squeeze(np.array(Linfinity_test_error_list))
    Linfinity_test_error_list_mean = np.mean(Linfinity_test_error_list)
    Linfinity_norm_list_20trial_mean.append(Linfinity_test_error_list_mean)


    # 💎logging in process "for trial_i in range(10, 20)"
    with open(log_filename, 'a') as log_file:
        log_file.write(
            f'------------------------------------------------- train_size: {train_size} ------------------------------------------------- \n'
            f'history:\nL2_C_20trial: {L2_C_20trial}\n'
            f'L2_t_list_20trial: {L2_t_list_20trial}\n'
            f'L2_norm_list_20trial: {L2_norm_list_20trial}\n'
            f'L2_norm_list_20trial_mean: {L2_norm_list_20trial_mean}\n'
            f'Linfinity_C_20trial: {Linfinity_C_20trial}\n'
            f'Linfinity_t_list_20trial: {Linfinity_t_list_20trial}\n'
            f'Linfinity_norm_list_20trial: {Linfinity_norm_list_20trial}\n'
            f'Linfinity_norm_list_20trial_mean: {Linfinity_norm_list_20trial_mean}\n'
        )

    # ☀️save in process
    KGD_result = {
        'L2_C_20trial': L2_C_20trial,
        'L2_t_list_20trial': L2_t_list_20trial,
        'L2_norm_list_20trial': L2_norm_list_20trial,
        'L2_norm_list_20trial_mean': L2_norm_list_20trial_mean,
        'Linfinity_C_20trial': Linfinity_C_20trial,
        'Linfinity_t_list_20trial': Linfinity_t_list_20trial,
        'Linfinity_norm_list_20trial': Linfinity_norm_list_20trial,
        'Linfinity_norm_list_20trial_mean': Linfinity_norm_list_20trial_mean}
    filename = f'./HSS_result_10trials/Small_Crange01new_HSStest_1000_dim{dim}_noisevar{noise_var}_split_L_per{split_L_per}_stepsize{step_size}_new.npy'
    np.save(filename, KGD_result)


time_total = time.time() - time_start

# ☀️save final
KGD_result = {
    'L2_C_20trial': L2_C_20trial,
    'L2_t_list_20trial': L2_t_list_20trial,
    'L2_norm_list_20trial': L2_norm_list_20trial,
    'L2_norm_list_20trial_mean': L2_norm_list_20trial_mean,
    'Linfinity_C_20trial': Linfinity_C_20trial,
    'Linfinity_t_list_20trial': Linfinity_t_list_20trial,
    'Linfinity_norm_list_20trial': Linfinity_norm_list_20trial,
    'Linfinity_norm_list_20trial_mean': Linfinity_norm_list_20trial_mean}
filename = f'./HSS_result_10trials/Small_Crange01new_HSStest_1000_dim{dim}_noisevar{noise_var}_split_L_per{split_L_per}_stepsize{step_size}_new.npy'
np.save(filename, KGD_result)
with open(log_filename, 'a') as log_file:
    log_file.write(f'Total running time: {time_total}\n')


print('------------------------------ runing information ------------------------------------------')
print('running time:', time_total)
print(filename)
print(KGD_result.keys())
print(log_filename)
print("Logging complete.")




In [ ]:
filename = f'./HSS_result_10trials/Small_Crange01new_HSStest_1000_dim{dim}_noisevar{noise_var}_split_L_per{split_L_per}_stepsize{step_size}_new.npy'
HSS_result = np.load(filename, allow_pickle=True).item()
HSS_L2_norm_list_20trial_mean = HSS_result['L2_norm_list_20trial_mean']
HSS_Linfinity_norm_list_20trial_mean = HSS_result['Linfinity_norm_list_20trial_mean']
np.set_printoptions(suppress=True, precision=6)
print(HSS_L2_norm_list_20trial_mean)
print(HSS_result['L2_C_20trial'])
print(HSS_Linfinity_norm_list_20trial_mean)
print(HSS_result['Linfinity_C_20trial'])

In [ ]:
import pandas as pd
from numpy import *
import numpy as np
from HSS_test_compare_BP import generate_data, Test_error_KGD_BSP
import math
import h5py
from sklearn.model_selection import KFold
import os, time
from datetime import datetime
time_start = time.time()
import random
import matplotlib.pyplot as plt
start_time = datetime.now()
start_time_str = start_time.strftime('%Y.%m.%d')

split_L_tr_per = 0.7 # 固定不变
noise_var = 0.6



dim = 1
step_size = 1
split_L_per = 0.6 # 💎1， 0.8，0.6， 0.4
C_bsp_list = [0.00001, 0.0001, 0.001, 0.01] + [i * 0.05 for i in range(1, 21, 1)]  # d=1

log_filename = f'./HSS_result_10trials/Small_Crange01new_HSStest_1200_dim{dim}_noisevar{noise_var}_split_L_per{split_L_per}_stepsize{step_size}_start{start_time_str}_log.txt'
# 初始化日志文件
with open(log_filename, 'w') as log_file:
    log_file.write("Start Logging\n")


# L2_C_20trial, L2_t_list_20trial, L2_norm_list_20trial = np.empty(shape=(10, 15)), np.empty(shape=(10, 15)), np.empty(shape=(10, 15))
# Linfinity_C_20trial, Linfinity_t_list_20trial, Linfinity_norm_list_20trial = np.empty(shape=(10, 15)), np.empty(shape=(10, 15)), np.empty(shape=(10, 15))
L2_C_20trial, L2_t_list_20trial, L2_norm_list_20trial = np.empty(shape=(10, 1)), np.empty(shape=(10, 1)), np.empty(shape=(10, 1))
Linfinity_C_20trial, Linfinity_t_list_20trial, Linfinity_norm_list_20trial = np.empty(shape=(10, 1)), np.empty(shape=(10, 1)), np.empty(shape=(10, 1))
L2_norm_list_20trial_mean, Linfinity_norm_list_20trial_mean = [], []


for size_i in range(1,2):
    L2_C_list, L2_t_star_list, L2_test_error_list, Linfinity_C_list, Linfinity_t_star_list, Linfinity_test_error_list = [], [], [], [], [], []
    train_size = 1200 * size_i
    split_L = int(train_size * split_L_per)
    split_L_tr = int(split_L * split_L_tr_per)
    train, test = (train_size, dim), (200, dim)
    print(' ------------------  💎 dim: %s; step_size: %s; train_size: %s; split_L_per: %s; split_L_tr_per: %s; noise_var: %s  ------------------' % (dim, step_size, train_size, split_L_per, split_L_tr_per, noise_var))

    for trial_i in range(15, 25):
        print('------------------   trial_i: %s  ------------------' % (trial_i-9))
        np.random.seed(trial_i)
        _, _, X_test, y_test = generate_data((300, dim), (500, dim), dim, noise_var)  #保证 test只在不同的trial时有所不同
        X_train, y_train, _, _ = generate_data(train, test, dim, noise_var)
        L2_C, L2_t_star, L2_test_error, Linfinity_C, Linfinity_t_star, Linfinity_test_error = Test_error_KGD_BSP(X_train, y_train, X_test, y_test, dim, step_size, split_L, split_L_tr, log_filename, C_bsp_list)
        L2_C_list.append(L2_C)
        L2_t_star_list.append(L2_t_star)
        L2_test_error_list.append(L2_test_error)
        Linfinity_C_list.append(Linfinity_C)
        Linfinity_t_star_list.append(Linfinity_t_star)
        Linfinity_test_error_list.append(Linfinity_test_error)
        print('L2_t_star_list:', L2_t_star_list)
        print('L2_test_error_list:', L2_test_error_list)
        print('Linfinity_t_star_list:', Linfinity_t_star_list)
        print('Linfinity_test_error_list:', Linfinity_test_error_list)


        # 💎logging in process "for trial_i in range(10, 20)"
        with open(log_filename, 'a') as log_file:
            log_file.write(
                f'--------------------------- train_size: {train_size}; trial_i: {trial_i-10}--------------------------- \n'
                f'history:\nL2_C_list: {L2_C_list}\n'
                f'L2_t_star_list: {L2_t_star_list}\n'
                f'L2_test_error_list: {L2_test_error_list}\n'
                f'Linfinity_C_list: {Linfinity_C_list}\n'
                f'Linfinity_t_star_list: {Linfinity_t_star_list}\n'
                f'Linfinity_test_error_list: {Linfinity_test_error_list}\n')


    L2_C_20trial[:, size_i-1] = np.squeeze(np.array(L2_C_list))
    L2_t_list_20trial[:, size_i - 1] = np.squeeze(np.array(L2_t_star_list))
    L2_norm_list_20trial[:, size_i - 1] = np.squeeze(np.array(L2_test_error_list))
    L2_test_error_list_mean = np.mean(L2_test_error_list)
    L2_norm_list_20trial_mean.append(L2_test_error_list_mean)

    Linfinity_C_20trial[:, size_i-1] = np.squeeze(np.array(Linfinity_C_list))
    Linfinity_t_list_20trial[:, size_i - 1] = np.squeeze(np.array(Linfinity_t_star_list))
    Linfinity_norm_list_20trial[:, size_i - 1] = np.squeeze(np.array(Linfinity_test_error_list))
    Linfinity_test_error_list_mean = np.mean(Linfinity_test_error_list)
    Linfinity_norm_list_20trial_mean.append(Linfinity_test_error_list_mean)


    # 💎logging in process "for trial_i in range(10, 20)"
    with open(log_filename, 'a') as log_file:
        log_file.write(
            f'------------------------------------------------- train_size: {train_size} ------------------------------------------------- \n'
            f'history:\nL2_C_20trial: {L2_C_20trial}\n'
            f'L2_t_list_20trial: {L2_t_list_20trial}\n'
            f'L2_norm_list_20trial: {L2_norm_list_20trial}\n'
            f'L2_norm_list_20trial_mean: {L2_norm_list_20trial_mean}\n'
            f'Linfinity_C_20trial: {Linfinity_C_20trial}\n'
            f'Linfinity_t_list_20trial: {Linfinity_t_list_20trial}\n'
            f'Linfinity_norm_list_20trial: {Linfinity_norm_list_20trial}\n'
            f'Linfinity_norm_list_20trial_mean: {Linfinity_norm_list_20trial_mean}\n'
        )

    # ☀️save in process
    KGD_result = {
        'L2_C_20trial': L2_C_20trial,
        'L2_t_list_20trial': L2_t_list_20trial,
        'L2_norm_list_20trial': L2_norm_list_20trial,
        'L2_norm_list_20trial_mean': L2_norm_list_20trial_mean,
        'Linfinity_C_20trial': Linfinity_C_20trial,
        'Linfinity_t_list_20trial': Linfinity_t_list_20trial,
        'Linfinity_norm_list_20trial': Linfinity_norm_list_20trial,
        'Linfinity_norm_list_20trial_mean': Linfinity_norm_list_20trial_mean}
    filename = f'./HSS_result_10trials/Small_Crange01new_HSStest_1200_dim{dim}_noisevar{noise_var}_split_L_per{split_L_per}_stepsize{step_size}_new.npy'
    np.save(filename, KGD_result)


time_total = time.time() - time_start

# ☀️save final
KGD_result = {
    'L2_C_20trial': L2_C_20trial,
    'L2_t_list_20trial': L2_t_list_20trial,
    'L2_norm_list_20trial': L2_norm_list_20trial,
    'L2_norm_list_20trial_mean': L2_norm_list_20trial_mean,
    'Linfinity_C_20trial': Linfinity_C_20trial,
    'Linfinity_t_list_20trial': Linfinity_t_list_20trial,
    'Linfinity_norm_list_20trial': Linfinity_norm_list_20trial,
    'Linfinity_norm_list_20trial_mean': Linfinity_norm_list_20trial_mean}
filename = f'./HSS_result_10trials/Small_Crange01new_HSStest_1200_dim{dim}_noisevar{noise_var}_split_L_per{split_L_per}_stepsize{step_size}_new.npy'
np.save(filename, KGD_result)
with open(log_filename, 'a') as log_file:
    log_file.write(f'Total running time: {time_total}\n')


print('------------------------------ runing information ------------------------------------------')
print('running time:', time_total)
print(filename)
print(KGD_result.keys())
print(log_filename)
print("Logging complete.")




In [2]:
import pandas as pd
from numpy import *
import numpy as np
from HSS_test_compare_BP import generate_data, Test_error_KGD_BSP
import math
import h5py
from sklearn.model_selection import KFold
import os, time
from datetime import datetime
time_start = time.time()
import random
import matplotlib.pyplot as plt
start_time = datetime.now()
start_time_str = start_time.strftime('%Y.%m.%d')

split_L_tr_per = 0.7 # 固定不变
noise_var = 0.6



dim = 1
step_size = 1
split_L_per = 0.6 # 💎1， 0.8，0.6， 0.4
C_bsp_list = [0.00001, 0.0001, 0.001, 0.01] + [i * 0.05 for i in range(1, 21, 1)]  # d=1
filename = f'./HSS_result_10trials/Small_Crange01new_HSStest_1200_dim{dim}_noisevar{noise_var}_split_L_per{split_L_per}_stepsize{step_size}_new.npy'
HSS_result = np.load(filename, allow_pickle=True).item()
HSS_L2_norm_list_20trial_mean = HSS_result['L2_norm_list_20trial_mean']
HSS_Linfinity_norm_list_20trial_mean = HSS_result['Linfinity_norm_list_20trial_mean']
np.set_printoptions(suppress=True, precision=6)
print(HSS_L2_norm_list_20trial_mean)
print(HSS_result['L2_C_20trial'])
print(HSS_Linfinity_norm_list_20trial_mean)
print(HSS_result['Linfinity_C_20trial'])

FileNotFoundError: [Errno 2] No such file or directory: './HSS_result_10trials/Small_Crange01new_HSStest_1200_dim1_noisevar0.6_split_L_per0.6_stepsize1_new.npy'

In [ ]:
import pandas as pd
from numpy import *
import numpy as np
from HSS_test_compare_BP import generate_data, Test_error_KGD_BSP
import math
import h5py
from sklearn.model_selection import KFold
import os, time
from datetime import datetime
time_start = time.time()
import random
import matplotlib.pyplot as plt
start_time = datetime.now()
start_time_str = start_time.strftime('%Y.%m.%d')

split_L_tr_per = 0.7 # 固定不变
noise_var = 0.6



dim = 3
step_size = 3
split_L_per = 1.0 # 💎1， 0.8，0.6， 0.4
C_bsp_list = [0.00001, 0.0001, 0.001, 0.01] + [i * 0.05 for i in range(1, 21, 1)]  # d=1

log_filename = f'./HSS_result_10trials/Small_Crange01new_HSStest_1000_dim{dim}_noisevar{noise_var}_split_L_per{split_L_per}_stepsize{step_size}_start{start_time_str}_log.txt'
# 初始化日志文件
with open(log_filename, 'w') as log_file:
    log_file.write("Start Logging\n")


# L2_C_20trial, L2_t_list_20trial, L2_norm_list_20trial = np.empty(shape=(10, 15)), np.empty(shape=(10, 15)), np.empty(shape=(10, 15))
# Linfinity_C_20trial, Linfinity_t_list_20trial, Linfinity_norm_list_20trial = np.empty(shape=(10, 15)), np.empty(shape=(10, 15)), np.empty(shape=(10, 15))
L2_C_20trial, L2_t_list_20trial, L2_norm_list_20trial = np.empty(shape=(10, 1)), np.empty(shape=(10, 1)), np.empty(shape=(10, 1))
Linfinity_C_20trial, Linfinity_t_list_20trial, Linfinity_norm_list_20trial = np.empty(shape=(10, 1)), np.empty(shape=(10, 1)), np.empty(shape=(10, 1))
L2_norm_list_20trial_mean, Linfinity_norm_list_20trial_mean = [], []


for size_i in range(1,2):
    L2_C_list, L2_t_star_list, L2_test_error_list, Linfinity_C_list, Linfinity_t_star_list, Linfinity_test_error_list = [], [], [], [], [], []
    train_size = 1000 * size_i
    split_L = int(train_size * split_L_per)
    split_L_tr = int(split_L * split_L_tr_per)
    train, test = (train_size, dim), (200, dim)
    print(' ------------------  💎 dim: %s; step_size: %s; train_size: %s; split_L_per: %s; split_L_tr_per: %s; noise_var: %s  ------------------' % (dim, step_size, train_size, split_L_per, split_L_tr_per, noise_var))

    for trial_i in range(15, 25):
        print('------------------   trial_i: %s  ------------------' % (trial_i-9))
        np.random.seed(trial_i)
        _, _, X_test, y_test = generate_data((300, dim), (500, dim), dim, noise_var)  #保证 test只在不同的trial时有所不同
        X_train, y_train, _, _ = generate_data(train, test, dim, noise_var)
        L2_C, L2_t_star, L2_test_error, Linfinity_C, Linfinity_t_star, Linfinity_test_error = Test_error_KGD_BSP(X_train, y_train, X_test, y_test, dim, step_size, split_L, split_L_tr, log_filename, C_bsp_list)
        L2_C_list.append(L2_C)
        L2_t_star_list.append(L2_t_star)
        L2_test_error_list.append(L2_test_error)
        Linfinity_C_list.append(Linfinity_C)
        Linfinity_t_star_list.append(Linfinity_t_star)
        Linfinity_test_error_list.append(Linfinity_test_error)
        print('L2_t_star_list:', L2_t_star_list)
        print('L2_test_error_list:', L2_test_error_list)
        print('Linfinity_t_star_list:', Linfinity_t_star_list)
        print('Linfinity_test_error_list:', Linfinity_test_error_list)


        # 💎logging in process "for trial_i in range(10, 20)"
        with open(log_filename, 'a') as log_file:
            log_file.write(
                f'--------------------------- train_size: {train_size}; trial_i: {trial_i-10}--------------------------- \n'
                f'history:\nL2_C_list: {L2_C_list}\n'
                f'L2_t_star_list: {L2_t_star_list}\n'
                f'L2_test_error_list: {L2_test_error_list}\n'
                f'Linfinity_C_list: {Linfinity_C_list}\n'
                f'Linfinity_t_star_list: {Linfinity_t_star_list}\n'
                f'Linfinity_test_error_list: {Linfinity_test_error_list}\n')


    L2_C_20trial[:, size_i-1] = np.squeeze(np.array(L2_C_list))
    L2_t_list_20trial[:, size_i - 1] = np.squeeze(np.array(L2_t_star_list))
    L2_norm_list_20trial[:, size_i - 1] = np.squeeze(np.array(L2_test_error_list))
    L2_test_error_list_mean = np.mean(L2_test_error_list)
    L2_norm_list_20trial_mean.append(L2_test_error_list_mean)

    Linfinity_C_20trial[:, size_i-1] = np.squeeze(np.array(Linfinity_C_list))
    Linfinity_t_list_20trial[:, size_i - 1] = np.squeeze(np.array(Linfinity_t_star_list))
    Linfinity_norm_list_20trial[:, size_i - 1] = np.squeeze(np.array(Linfinity_test_error_list))
    Linfinity_test_error_list_mean = np.mean(Linfinity_test_error_list)
    Linfinity_norm_list_20trial_mean.append(Linfinity_test_error_list_mean)


    # 💎logging in process "for trial_i in range(10, 20)"
    with open(log_filename, 'a') as log_file:
        log_file.write(
            f'------------------------------------------------- train_size: {train_size} ------------------------------------------------- \n'
            f'history:\nL2_C_20trial: {L2_C_20trial}\n'
            f'L2_t_list_20trial: {L2_t_list_20trial}\n'
            f'L2_norm_list_20trial: {L2_norm_list_20trial}\n'
            f'L2_norm_list_20trial_mean: {L2_norm_list_20trial_mean}\n'
            f'Linfinity_C_20trial: {Linfinity_C_20trial}\n'
            f'Linfinity_t_list_20trial: {Linfinity_t_list_20trial}\n'
            f'Linfinity_norm_list_20trial: {Linfinity_norm_list_20trial}\n'
            f'Linfinity_norm_list_20trial_mean: {Linfinity_norm_list_20trial_mean}\n'
        )

    # ☀️save in process
    KGD_result = {
        'L2_C_20trial': L2_C_20trial,
        'L2_t_list_20trial': L2_t_list_20trial,
        'L2_norm_list_20trial': L2_norm_list_20trial,
        'L2_norm_list_20trial_mean': L2_norm_list_20trial_mean,
        'Linfinity_C_20trial': Linfinity_C_20trial,
        'Linfinity_t_list_20trial': Linfinity_t_list_20trial,
        'Linfinity_norm_list_20trial': Linfinity_norm_list_20trial,
        'Linfinity_norm_list_20trial_mean': Linfinity_norm_list_20trial_mean}
    filename = f'./HSS_result_10trials/Small_Crange01new_HSStest_1000_dim{dim}_noisevar{noise_var}_split_L_per{split_L_per}_stepsize{step_size}_new.npy'
    np.save(filename, KGD_result)


time_total = time.time() - time_start

# ☀️save final
KGD_result = {
    'L2_C_20trial': L2_C_20trial,
    'L2_t_list_20trial': L2_t_list_20trial,
    'L2_norm_list_20trial': L2_norm_list_20trial,
    'L2_norm_list_20trial_mean': L2_norm_list_20trial_mean,
    'Linfinity_C_20trial': Linfinity_C_20trial,
    'Linfinity_t_list_20trial': Linfinity_t_list_20trial,
    'Linfinity_norm_list_20trial': Linfinity_norm_list_20trial,
    'Linfinity_norm_list_20trial_mean': Linfinity_norm_list_20trial_mean}
filename = f'./HSS_result_10trials/Small_Crange01new_HSStest_1000_dim{dim}_noisevar{noise_var}_split_L_per{split_L_per}_stepsize{step_size}_new.npy'
np.save(filename, KGD_result)
with open(log_filename, 'a') as log_file:
    log_file.write(f'Total running time: {time_total}\n')


print('------------------------------ runing information ------------------------------------------')
print('running time:', time_total)
print(filename)
print(KGD_result.keys())
print(log_filename)
print("Logging complete.")




In [ ]:
filename = f'./HSS_result_10trials/Small_Crange01new_HSStest_1000_dim{dim}_noisevar{noise_var}_split_L_per{split_L_per}_stepsize{step_size}_new.npy'
HSS_result = np.load(filename, allow_pickle=True).item()
HSS_L2_norm_list_20trial_mean = HSS_result['L2_norm_list_20trial_mean']
HSS_Linfinity_norm_list_20trial_mean = HSS_result['Linfinity_norm_list_20trial_mean']
np.set_printoptions(suppress=True, precision=6)
print(HSS_L2_norm_list_20trial_mean)
print(HSS_result['L2_C_20trial'])
print(HSS_Linfinity_norm_list_20trial_mean)
print(HSS_result['Linfinity_C_20trial'])

In [ ]:
import pandas as pd
from numpy import *
import numpy as np
from HSS_test_compare_BP import generate_data, Test_error_KGD_BSP
import math
import h5py
from sklearn.model_selection import KFold
import os, time
from datetime import datetime
time_start = time.time()
import random
import matplotlib.pyplot as plt
start_time = datetime.now()
start_time_str = start_time.strftime('%Y.%m.%d')

split_L_tr_per = 0.7 # 固定不变
noise_var = 0.6



dim = 3
step_size = 3
split_L_per = 1.0 # 💎1， 0.8，0.6， 0.4
C_bsp_list = [0.00001, 0.0001, 0.001, 0.01] + [i * 0.05 for i in range(1, 21, 1)]  # d=1

log_filename = f'./HSS_result_10trials/Small_Crange01new_HSStest_1200_dim{dim}_noisevar{noise_var}_split_L_per{split_L_per}_stepsize{step_size}_start{start_time_str}_log.txt'
# 初始化日志文件
with open(log_filename, 'w') as log_file:
    log_file.write("Start Logging\n")


# L2_C_20trial, L2_t_list_20trial, L2_norm_list_20trial = np.empty(shape=(10, 15)), np.empty(shape=(10, 15)), np.empty(shape=(10, 15))
# Linfinity_C_20trial, Linfinity_t_list_20trial, Linfinity_norm_list_20trial = np.empty(shape=(10, 15)), np.empty(shape=(10, 15)), np.empty(shape=(10, 15))
L2_C_20trial, L2_t_list_20trial, L2_norm_list_20trial = np.empty(shape=(10, 1)), np.empty(shape=(10, 1)), np.empty(shape=(10, 1))
Linfinity_C_20trial, Linfinity_t_list_20trial, Linfinity_norm_list_20trial = np.empty(shape=(10, 1)), np.empty(shape=(10, 1)), np.empty(shape=(10, 1))
L2_norm_list_20trial_mean, Linfinity_norm_list_20trial_mean = [], []


for size_i in range(1,2):
    L2_C_list, L2_t_star_list, L2_test_error_list, Linfinity_C_list, Linfinity_t_star_list, Linfinity_test_error_list = [], [], [], [], [], []
    train_size = 1200 * size_i
    split_L = int(train_size * split_L_per)
    split_L_tr = int(split_L * split_L_tr_per)
    train, test = (train_size, dim), (200, dim)
    print(' ------------------  💎 dim: %s; step_size: %s; train_size: %s; split_L_per: %s; split_L_tr_per: %s; noise_var: %s  ------------------' % (dim, step_size, train_size, split_L_per, split_L_tr_per, noise_var))

    for trial_i in range(15, 25):
        print('------------------   trial_i: %s  ------------------' % (trial_i-9))
        np.random.seed(trial_i)
        _, _, X_test, y_test = generate_data((300, dim), (500, dim), dim, noise_var)  #保证 test只在不同的trial时有所不同
        X_train, y_train, _, _ = generate_data(train, test, dim, noise_var)
        L2_C, L2_t_star, L2_test_error, Linfinity_C, Linfinity_t_star, Linfinity_test_error = Test_error_KGD_BSP(X_train, y_train, X_test, y_test, dim, step_size, split_L, split_L_tr, log_filename, C_bsp_list)
        L2_C_list.append(L2_C)
        L2_t_star_list.append(L2_t_star)
        L2_test_error_list.append(L2_test_error)
        Linfinity_C_list.append(Linfinity_C)
        Linfinity_t_star_list.append(Linfinity_t_star)
        Linfinity_test_error_list.append(Linfinity_test_error)
        print('L2_t_star_list:', L2_t_star_list)
        print('L2_test_error_list:', L2_test_error_list)
        print('Linfinity_t_star_list:', Linfinity_t_star_list)
        print('Linfinity_test_error_list:', Linfinity_test_error_list)


        # 💎logging in process "for trial_i in range(10, 20)"
        with open(log_filename, 'a') as log_file:
            log_file.write(
                f'--------------------------- train_size: {train_size}; trial_i: {trial_i-10}--------------------------- \n'
                f'history:\nL2_C_list: {L2_C_list}\n'
                f'L2_t_star_list: {L2_t_star_list}\n'
                f'L2_test_error_list: {L2_test_error_list}\n'
                f'Linfinity_C_list: {Linfinity_C_list}\n'
                f'Linfinity_t_star_list: {Linfinity_t_star_list}\n'
                f'Linfinity_test_error_list: {Linfinity_test_error_list}\n')


    L2_C_20trial[:, size_i-1] = np.squeeze(np.array(L2_C_list))
    L2_t_list_20trial[:, size_i - 1] = np.squeeze(np.array(L2_t_star_list))
    L2_norm_list_20trial[:, size_i - 1] = np.squeeze(np.array(L2_test_error_list))
    L2_test_error_list_mean = np.mean(L2_test_error_list)
    L2_norm_list_20trial_mean.append(L2_test_error_list_mean)

    Linfinity_C_20trial[:, size_i-1] = np.squeeze(np.array(Linfinity_C_list))
    Linfinity_t_list_20trial[:, size_i - 1] = np.squeeze(np.array(Linfinity_t_star_list))
    Linfinity_norm_list_20trial[:, size_i - 1] = np.squeeze(np.array(Linfinity_test_error_list))
    Linfinity_test_error_list_mean = np.mean(Linfinity_test_error_list)
    Linfinity_norm_list_20trial_mean.append(Linfinity_test_error_list_mean)


    # 💎logging in process "for trial_i in range(10, 20)"
    with open(log_filename, 'a') as log_file:
        log_file.write(
            f'------------------------------------------------- train_size: {train_size} ------------------------------------------------- \n'
            f'history:\nL2_C_20trial: {L2_C_20trial}\n'
            f'L2_t_list_20trial: {L2_t_list_20trial}\n'
            f'L2_norm_list_20trial: {L2_norm_list_20trial}\n'
            f'L2_norm_list_20trial_mean: {L2_norm_list_20trial_mean}\n'
            f'Linfinity_C_20trial: {Linfinity_C_20trial}\n'
            f'Linfinity_t_list_20trial: {Linfinity_t_list_20trial}\n'
            f'Linfinity_norm_list_20trial: {Linfinity_norm_list_20trial}\n'
            f'Linfinity_norm_list_20trial_mean: {Linfinity_norm_list_20trial_mean}\n'
        )

    # ☀️save in process
    KGD_result = {
        'L2_C_20trial': L2_C_20trial,
        'L2_t_list_20trial': L2_t_list_20trial,
        'L2_norm_list_20trial': L2_norm_list_20trial,
        'L2_norm_list_20trial_mean': L2_norm_list_20trial_mean,
        'Linfinity_C_20trial': Linfinity_C_20trial,
        'Linfinity_t_list_20trial': Linfinity_t_list_20trial,
        'Linfinity_norm_list_20trial': Linfinity_norm_list_20trial,
        'Linfinity_norm_list_20trial_mean': Linfinity_norm_list_20trial_mean}
    filename = f'./HSS_result_10trials/Small_Crange01new_HSStest_1200_dim{dim}_noisevar{noise_var}_split_L_per{split_L_per}_stepsize{step_size}_new.npy'
    np.save(filename, KGD_result)


time_total = time.time() - time_start

# ☀️save final
KGD_result = {
    'L2_C_20trial': L2_C_20trial,
    'L2_t_list_20trial': L2_t_list_20trial,
    'L2_norm_list_20trial': L2_norm_list_20trial,
    'L2_norm_list_20trial_mean': L2_norm_list_20trial_mean,
    'Linfinity_C_20trial': Linfinity_C_20trial,
    'Linfinity_t_list_20trial': Linfinity_t_list_20trial,
    'Linfinity_norm_list_20trial': Linfinity_norm_list_20trial,
    'Linfinity_norm_list_20trial_mean': Linfinity_norm_list_20trial_mean}
filename = f'./HSS_result_10trials/Small_Crange01new_HSStest_1200_dim{dim}_noisevar{noise_var}_split_L_per{split_L_per}_stepsize{step_size}_new.npy'
np.save(filename, KGD_result)
with open(log_filename, 'a') as log_file:
    log_file.write(f'Total running time: {time_total}\n')


print('------------------------------ runing information ------------------------------------------')
print('running time:', time_total)
print(filename)
print(KGD_result.keys())
print(log_filename)
print("Logging complete.")




In [ ]:
filename = f'./HSS_result_10trials/Small_Crange01new_HSStest_1200_dim{dim}_noisevar{noise_var}_split_L_per{split_L_per}_stepsize{step_size}_new.npy'
HSS_result = np.load(filename, allow_pickle=True).item()
HSS_L2_norm_list_20trial_mean = HSS_result['L2_norm_list_20trial_mean']
HSS_Linfinity_norm_list_20trial_mean = HSS_result['Linfinity_norm_list_20trial_mean']
np.set_printoptions(suppress=True, precision=6)
print(HSS_L2_norm_list_20trial_mean)
print(HSS_result['L2_C_20trial'])
print(HSS_Linfinity_norm_list_20trial_mean)
print(HSS_result['Linfinity_C_20trial'])

In [ ]:
import pandas as pd
from numpy import *
import numpy as np
from LP_original import generate_data, Test_error_KGD_LP
import math
import h5py
import os, time
from datetime import datetime
time_start = time.time()
import random
start_time = datetime.now()
start_time_str = start_time.strftime('%Y.%m.%d')

split_L_tr_per = 0.7 # 固定不变
noise_var = 0.6



dim = 1
step_size = 1
split_L_per = 0.6 # 💎1， 0.8，0.6， 0.4
C_bsp_list = [0.00001, 0.0001, 0.001, 0.01] + [i * 0.05 for i in range(1, 21, 1)]  # d=1

log_filename = f'./LP_result_10trials/small_Crange01_LP_1000_dim{dim}_noisevar{noise_var}_split_L_per{split_L_per}_stepsize{step_size}_start{start_time_str}_log.txt'
# 初始化日志文件
with open(log_filename, 'w') as log_file:
    log_file.write("Start Logging\n")


# L2_C_20trial, L2_t_list_20trial, L2_norm_list_20trial = np.empty(shape=(10, 15)), np.empty(shape=(10, 15)), np.empty(shape=(10, 15))
# Linfinity_C_20trial, Linfinity_t_list_20trial, Linfinity_norm_list_20trial = np.empty(shape=(10, 15)), np.empty(shape=(10, 15)), np.empty(shape=(10, 15))
L2_C_20trial, L2_t_list_20trial, L2_norm_list_20trial = np.empty(shape=(10, 1)), np.empty(shape=(10, 1)), np.empty(shape=(10, 1))
Linfinity_C_20trial, Linfinity_t_list_20trial, Linfinity_norm_list_20trial = np.empty(shape=(10, 1)), np.empty(shape=(10, 1)), np.empty(shape=(10, 1))
L2_norm_list_20trial_mean, Linfinity_norm_list_20trial_mean = [], []


for size_i in range(1, 2):
    L2_C_list, L2_t_star_list, L2_test_error_list, Linfinity_C_list, Linfinity_t_star_list, Linfinity_test_error_list = [], [], [], [], [], []
    train_size = 1000 * size_i
    split_L = int(train_size * split_L_per)
    split_L_tr = int(split_L * split_L_tr_per)
    train, test = (train_size, dim), (200, dim)
    print(' ------------------  💎 dim: %s; step_size: %s; train_size: %s; split_L_per: %s; split_L_tr_per: %s; noise_var: %s  ------------------' % (dim, step_size, train_size, split_L, split_L_tr, noise_var))

    for trial_i in range(15, 25):
        print('------------------   trial_i: %s  ------------------' % (trial_i-9))
        np.random.seed(trial_i)
        _, _, X_test, y_test = generate_data((300, dim), (500, dim), dim, noise_var)  #保证 test只在不同的trial时有所不同
        X_train, y_train, _, _ = generate_data(train, test, dim, noise_var)
        L2_C, L2_t_star, L2_test_error, Linfinity_C, Linfinity_t_star, Linfinity_test_error = Test_error_KGD_LP(X_train, y_train, X_test, y_test, dim, step_size, split_L, split_L_tr, log_filename, C_bsp_list)
        L2_C_list.append(L2_C)
        L2_t_star_list.append(L2_t_star)
        L2_test_error_list.append(L2_test_error)
        Linfinity_C_list.append(Linfinity_C)
        Linfinity_t_star_list.append(Linfinity_t_star)
        Linfinity_test_error_list.append(Linfinity_test_error)
        print('L2_t_star_list:', L2_t_star_list)
        print('L2_test_error_list:', L2_test_error_list)
        print('Linfinity_t_star_list:', Linfinity_t_star_list)
        print('Linfinity_test_error_list:', Linfinity_test_error_list)


        # 💎logging in process "for trial_i in range(10, 20)"
        with open(log_filename, 'a') as log_file:
            log_file.write(
                f'--------------------------- train_size: {train_size}; trial_i: {trial_i-10}--------------------------- \n'
                f'history:\nL2_C_list: {L2_C_list}\n'
                f'L2_t_star_list: {L2_t_star_list}\n'
                f'L2_test_error_list: {L2_test_error_list}\n'
                f'Linfinity_C_list: {Linfinity_C_list}\n'
                f'Linfinity_t_star_list: {Linfinity_t_star_list}\n'
                f'Linfinity_test_error_list: {Linfinity_test_error_list}\n')


    L2_C_20trial[:, size_i-1] = np.squeeze(np.array(L2_C_list))
    L2_t_list_20trial[:, size_i - 1] = np.squeeze(np.array(L2_t_star_list))
    L2_norm_list_20trial[:, size_i - 1] = np.squeeze(np.array(L2_test_error_list))
    L2_test_error_list_mean = np.mean(L2_test_error_list)
    L2_norm_list_20trial_mean.append(L2_test_error_list_mean)

    Linfinity_C_20trial[:, size_i-1] = np.squeeze(np.array(Linfinity_C_list))
    Linfinity_t_list_20trial[:, size_i - 1] = np.squeeze(np.array(Linfinity_t_star_list))
    Linfinity_norm_list_20trial[:, size_i - 1] = np.squeeze(np.array(Linfinity_test_error_list))
    Linfinity_test_error_list_mean = np.mean(Linfinity_test_error_list)
    Linfinity_norm_list_20trial_mean.append(Linfinity_test_error_list_mean)


    # 💎logging in process "for trial_i in range(10, 20)"
    with open(log_filename, 'a') as log_file:
        log_file.write(
            f'------------------------------------------------- train_size: {train_size} ------------------------------------------------- \n'
            f'history:\nL2_C_20trial: {L2_C_20trial}\n'
            f'L2_t_list_20trial: {L2_t_list_20trial}\n'
            f'L2_norm_list_20trial: {L2_norm_list_20trial}\n'
            f'L2_norm_list_20trial_mean: {L2_norm_list_20trial_mean}\n'
            f'Linfinity_C_20trial: {Linfinity_C_20trial}\n'
            f'Linfinity_t_list_20trial: {Linfinity_t_list_20trial}\n'
            f'Linfinity_norm_list_20trial: {Linfinity_norm_list_20trial}\n'
            f'Linfinity_norm_list_20trial_mean: {Linfinity_norm_list_20trial_mean}\n'
        )

    # ☀️save in process
    KGD_result = {
        'L2_C_20trial': L2_C_20trial,
        'L2_t_list_20trial': L2_t_list_20trial,
        'L2_norm_list_20trial': L2_norm_list_20trial,
        'L2_norm_list_20trial_mean': L2_norm_list_20trial_mean,
        'Linfinity_C_20trial': Linfinity_C_20trial,
        'Linfinity_t_list_20trial': Linfinity_t_list_20trial,
        'Linfinity_norm_list_20trial': Linfinity_norm_list_20trial,
        'Linfinity_norm_list_20trial_mean': Linfinity_norm_list_20trial_mean}
    filename = f'./LP_result_10trials/small_Crange01_LP_1000_dim{dim}_noisevar{noise_var}_split_L_per{split_L_per}_stepsize{step_size}_new.npy'
    np.save(filename, KGD_result)


time_total = time.time() - time_start

# ☀️save final
KGD_result = {
    'L2_C_20trial': L2_C_20trial,
    'L2_t_list_20trial': L2_t_list_20trial,
    'L2_norm_list_20trial': L2_norm_list_20trial,
    'L2_norm_list_20trial_mean': L2_norm_list_20trial_mean,
    'Linfinity_C_20trial': Linfinity_C_20trial,
    'Linfinity_t_list_20trial': Linfinity_t_list_20trial,
    'Linfinity_norm_list_20trial': Linfinity_norm_list_20trial,
    'Linfinity_norm_list_20trial_mean': Linfinity_norm_list_20trial_mean}
filename = f'./LP_result_10trials/small_Crange01_LP_1000_dim{dim}_noisevar{noise_var}_split_L_per{split_L_per}_stepsize{step_size}_new.npy'
np.save(filename, KGD_result)
with open(log_filename, 'a') as log_file:
    log_file.write(f'Total running time: {time_total}\n')


print('------------------------------ runing information ------------------------------------------')
print('running time:', time_total)
print(filename)
print(KGD_result.keys())
print(log_filename)
print("Logging complete.")


In [ ]:
filename = f'./LP_result_10trials/small_Crange01_LP_1000_dim{dim}_noisevar{noise_var}_split_L_per{split_L_per}_stepsize{step_size}_new.npy'
LP_result = np.load(filename, allow_pickle=True).item()
LP_L2_norm_list_20trial_mean = LP_result['L2_norm_list_20trial_mean']
LP_Linfinity_norm_list_20trial_mean = LP_result['Linfinity_norm_list_20trial_mean']
np.set_printoptions(suppress=True, precision=6)
print('LP with 1200 training size -------')
print(LP_L2_norm_list_20trial_mean)
print(LP_result['L2_C_20trial'])
print(LP_Linfinity_norm_list_20trial_mean)
print(LP_result['Linfinity_C_20trial'])

In [ ]:
import pandas as pd
from numpy import *
import numpy as np
from LP_speed import generate_data, Test_error_KGD_LP
import math
import h5py
import os, time
from datetime import datetime
time_start = time.time()
import random
start_time = datetime.now()
start_time_str = start_time.strftime('%Y.%m.%d')

split_L_tr_per = 0.7 # 固定不变
noise_var = 0.6



dim = 1
step_size = 1
split_L_per = 0.6 # 💎1， 0.8，0.6， 0.4
C_bsp_list = [0.00001, 0.0001, 0.001, 0.01] + [i * 0.05 for i in range(1, 21, 1)]  # d=1

log_filename = f'./LP_result_10trials/small_Crange01_LP_1000_dim{dim}_noisevar{noise_var}_split_L_per{split_L_per}_stepsize{step_size}_start{start_time_str}_log.txt'
# 初始化日志文件
with open(log_filename, 'w') as log_file:
    log_file.write("Start Logging\n")


# L2_C_20trial, L2_t_list_20trial, L2_norm_list_20trial = np.empty(shape=(10, 15)), np.empty(shape=(10, 15)), np.empty(shape=(10, 15))
# Linfinity_C_20trial, Linfinity_t_list_20trial, Linfinity_norm_list_20trial = np.empty(shape=(10, 15)), np.empty(shape=(10, 15)), np.empty(shape=(10, 15))
L2_C_20trial, L2_t_list_20trial, L2_norm_list_20trial = np.empty(shape=(10, 1)), np.empty(shape=(10, 1)), np.empty(shape=(10, 1))
Linfinity_C_20trial, Linfinity_t_list_20trial, Linfinity_norm_list_20trial = np.empty(shape=(10, 1)), np.empty(shape=(10, 1)), np.empty(shape=(10, 1))
L2_norm_list_20trial_mean, Linfinity_norm_list_20trial_mean = [], []


for size_i in range(1, 2):
    L2_C_list, L2_t_star_list, L2_test_error_list, Linfinity_C_list, Linfinity_t_star_list, Linfinity_test_error_list = [], [], [], [], [], []
    train_size = 1200 * size_i
    split_L = int(train_size * split_L_per)
    split_L_tr = int(split_L * split_L_tr_per)
    train, test = (train_size, dim), (200, dim)
    print(' ------------------  💎 dim: %s; step_size: %s; train_size: %s; split_L_per: %s; split_L_tr_per: %s; noise_var: %s  ------------------' % (dim, step_size, train_size, split_L, split_L_tr, noise_var))

    for trial_i in range(15, 25):
        print('------------------   trial_i: %s  ------------------' % (trial_i-9))
        np.random.seed(trial_i)
        _, _, X_test, y_test = generate_data((300, dim), (500, dim), dim, noise_var)  #保证 test只在不同的trial时有所不同
        X_train, y_train, _, _ = generate_data(train, test, dim, noise_var)
        L2_C, L2_t_star, L2_test_error, Linfinity_C, Linfinity_t_star, Linfinity_test_error = Test_error_KGD_LP(X_train, y_train, X_test, y_test, dim, step_size, split_L, split_L_tr, log_filename, C_bsp_list)
        L2_C_list.append(L2_C)
        L2_t_star_list.append(L2_t_star)
        L2_test_error_list.append(L2_test_error)
        Linfinity_C_list.append(Linfinity_C)
        Linfinity_t_star_list.append(Linfinity_t_star)
        Linfinity_test_error_list.append(Linfinity_test_error)
        print('L2_t_star_list:', L2_t_star_list)
        print('L2_test_error_list:', L2_test_error_list)
        print('Linfinity_t_star_list:', Linfinity_t_star_list)
        print('Linfinity_test_error_list:', Linfinity_test_error_list)


        # 💎logging in process "for trial_i in range(10, 20)"
        with open(log_filename, 'a') as log_file:
            log_file.write(
                f'--------------------------- train_size: {train_size}; trial_i: {trial_i-10}--------------------------- \n'
                f'history:\nL2_C_list: {L2_C_list}\n'
                f'L2_t_star_list: {L2_t_star_list}\n'
                f'L2_test_error_list: {L2_test_error_list}\n'
                f'Linfinity_C_list: {Linfinity_C_list}\n'
                f'Linfinity_t_star_list: {Linfinity_t_star_list}\n'
                f'Linfinity_test_error_list: {Linfinity_test_error_list}\n')


    L2_C_20trial[:, size_i-1] = np.squeeze(np.array(L2_C_list))
    L2_t_list_20trial[:, size_i - 1] = np.squeeze(np.array(L2_t_star_list))
    L2_norm_list_20trial[:, size_i - 1] = np.squeeze(np.array(L2_test_error_list))
    L2_test_error_list_mean = np.mean(L2_test_error_list)
    L2_norm_list_20trial_mean.append(L2_test_error_list_mean)

    Linfinity_C_20trial[:, size_i-1] = np.squeeze(np.array(Linfinity_C_list))
    Linfinity_t_list_20trial[:, size_i - 1] = np.squeeze(np.array(Linfinity_t_star_list))
    Linfinity_norm_list_20trial[:, size_i - 1] = np.squeeze(np.array(Linfinity_test_error_list))
    Linfinity_test_error_list_mean = np.mean(Linfinity_test_error_list)
    Linfinity_norm_list_20trial_mean.append(Linfinity_test_error_list_mean)


    # 💎logging in process "for trial_i in range(10, 20)"
    with open(log_filename, 'a') as log_file:
        log_file.write(
            f'------------------------------------------------- train_size: {train_size} ------------------------------------------------- \n'
            f'history:\nL2_C_20trial: {L2_C_20trial}\n'
            f'L2_t_list_20trial: {L2_t_list_20trial}\n'
            f'L2_norm_list_20trial: {L2_norm_list_20trial}\n'
            f'L2_norm_list_20trial_mean: {L2_norm_list_20trial_mean}\n'
            f'Linfinity_C_20trial: {Linfinity_C_20trial}\n'
            f'Linfinity_t_list_20trial: {Linfinity_t_list_20trial}\n'
            f'Linfinity_norm_list_20trial: {Linfinity_norm_list_20trial}\n'
            f'Linfinity_norm_list_20trial_mean: {Linfinity_norm_list_20trial_mean}\n'
        )

    # ☀️save in process
    KGD_result = {
        'L2_C_20trial': L2_C_20trial,
        'L2_t_list_20trial': L2_t_list_20trial,
        'L2_norm_list_20trial': L2_norm_list_20trial,
        'L2_norm_list_20trial_mean': L2_norm_list_20trial_mean,
        'Linfinity_C_20trial': Linfinity_C_20trial,
        'Linfinity_t_list_20trial': Linfinity_t_list_20trial,
        'Linfinity_norm_list_20trial': Linfinity_norm_list_20trial,
        'Linfinity_norm_list_20trial_mean': Linfinity_norm_list_20trial_mean}
    filename = f'./LP_result_10trials/small_Crange01_LP_1000_dim{dim}_noisevar{noise_var}_split_L_per{split_L_per}_stepsize{step_size}_new.npy'
    np.save(filename, KGD_result)


time_total = time.time() - time_start

# ☀️save final
KGD_result = {
    'L2_C_20trial': L2_C_20trial,
    'L2_t_list_20trial': L2_t_list_20trial,
    'L2_norm_list_20trial': L2_norm_list_20trial,
    'L2_norm_list_20trial_mean': L2_norm_list_20trial_mean,
    'Linfinity_C_20trial': Linfinity_C_20trial,
    'Linfinity_t_list_20trial': Linfinity_t_list_20trial,
    'Linfinity_norm_list_20trial': Linfinity_norm_list_20trial,
    'Linfinity_norm_list_20trial_mean': Linfinity_norm_list_20trial_mean}
filename = f'./LP_result_10trials/small_Crange01_LP_1000_dim{dim}_noisevar{noise_var}_split_L_per{split_L_per}_stepsize{step_size}_new.npy'
np.save(filename, KGD_result)
with open(log_filename, 'a') as log_file:
    log_file.write(f'Total running time: {time_total}\n')


print('------------------------------ runing information ------------------------------------------')
print('running time:', time_total)
print(filename)
print(KGD_result.keys())
print(log_filename)
print("Logging complete.")


In [ ]:
filename = f'./LP_result_10trials/small_Crange01_LP_1200_dim{dim}_noisevar{noise_var}_split_L_per{split_L_per}_stepsize{step_size}_new.npy'
LP_result = np.load(filename, allow_pickle=True).item()
LP_L2_norm_list_20trial_mean = LP_result['L2_norm_list_20trial_mean']
LP_Linfinity_norm_list_20trial_mean = LP_result['Linfinity_norm_list_20trial_mean']
np.set_printoptions(suppress=True, precision=6)
print('LP with 1200 training size -------')
print(LP_L2_norm_list_20trial_mean)
print(LP_result['L2_C_20trial'])
print(LP_Linfinity_norm_list_20trial_mean)
print(LP_result['Linfinity_C_20trial'])